# Preprocesamiento del dataset "Multistore"

In [1]:
import pandas as pd
import numpy as np
import os
import glob

## 1. Carga y concatenación de los archivos del dataset original

In [24]:
multistore_months = ['2019-Oct', '2019-Nov', '2019-Dec', '2020-Jan', '2020-Feb', '2020-Mar', '2020-Apr']

In [41]:
path = r'./' # cambiar por la ruta propia

def preprocess_month(month, path):
    month_file_r = os.path.join(path + month + '.csv')
    month_file_w = os.path.join(path + 'sales_multistore_' + month + '.csv')
    
    multistore_month_df = pd.read_csv(month_file_r)
    
    # Elimino las filas con nulos en las columnas 'category_code' y 'brand', porque no nos sirven sin esta información
    multistore_month_df =  multistore_month_df.dropna(subset=['category_code', 'brand'])
    
    # Obtención del subset de eventos de compra
    multistore_month_df =  multistore_month_df[ multistore_month_df['event_type'] == 'purchase']
    
    # Eliminación de la columna 'event_type'
    multistore_month_df =  multistore_month_df.drop('event_type', axis=1)

    multistore_month_df.to_csv(month_file_w, index=False)

In [26]:
# Creación de los csv "limpios"
for m in multistore_months:
    preprocess_month(m)

In [40]:
# Concatenación de los csv

all_files = glob.glob(os.path.join(path , "sales*.csv"))

multistore_sales_df = pd.concat((pd.read_csv(f) for f in all_files), ignore_index=True)    

In [38]:
multistore_sales_df.shape

(5707926, 8)

## 2. Limpieza previa de los datos

### 2.1 Filas duplicadas

In [8]:
multistore_sales_df[multistore_sales_df.duplicated()]

,event_time,product_id,category_id,category_code,brand,price,user_id,user_session
3310,2019-10-01 06:41:39 UTC,1004856,2053013555631882655,electronics.smartphone,samsung,130.76,555119233,53ad00ec-2d13-491c-9743-504913ea238c
9555,2019-10-01 12:42:40 UTC,1004767,2053013555631882655,electronics.smartphone,samsung,254.19,512386086,4a1f199d-e77b-4ff7-8807-47b31c1b6aca
49420,2019-10-04 07:20:16 UTC,5700758,2053013553970938175,auto.accessories.player,pioneer,75.94,517161433,80109315-6ff2-4b59-adec-f99cd75efb81
67687,2019-10-05 05:34:08 UTC,1004856,2053013555631882655,electronics.smartphone,samsung,131.92,535394720,af308845-2f01-43c0-a441-d70814450de6
79698,2019-10-05 16:37:05 UTC,1801690,2053013554415534427,electronics.video.tv,samsung,370.53,539724704,352cbf09-0d93-4086-9395-49605f75c59f
...,...,...,...,...,...,...,...,...
5297185,2020-04-16 17:00:09 UTC,24100688,2232732100954423843,appliances.personal.massager,kerasys,6.56,635946366,252d0d1f-23d9-4a36-bafd-82416840a9d1
5550832,2020-04-26 08:45:24 UTC,1002540,2232732093077520756,construction.tools.light,apple,409.78,515172019,5e8a156d-c481-4017-a550-63d4ba6ecaf8
5642867,2020-04-29 04:16:34 UTC,8800895,2053013560530830019,electronics.camera.video,bq,19.79,643856840,aa10f9d5-3275-4390-8b29-1e408b286f89
5666683,2020-04-29 15:33:22 UTC,1005169,2232732093077520756,construction.tools.light,samsung,180.42,648354166,2927cd4d-3a2a-4853-be26-f5139eff973f


¡Son muchas filas duplicadas! Espero que no haya ningún error en mi comando. Hago una comprobación y utilizo para ello el valor 'user_session' de la fila 3310.

In [12]:
# No me deja hacer una AND de la 'product_id' y la 'user_session'. Al parecer tienen que ser del mismo tipo
# de datos
multistore_sales_df.loc[multistore_sales_df['event_time'] == '2019-10-01 06:41:39 UTC']

,event_time,product_id,category_id,category_code,brand,price,user_id,user_session
3309,2019-10-01 06:41:39 UTC,1004856,2053013555631882655,electronics.smartphone,samsung,130.76,555119233,53ad00ec-2d13-491c-9743-504913ea238c
3310,2019-10-01 06:41:39 UTC,1004856,2053013555631882655,electronics.smartphone,samsung,130.76,555119233,53ad00ec-2d13-491c-9743-504913ea238c
3311,2019-10-01 06:41:39 UTC,12201514,2116907525176557699,sport.bicycle,giant,174.78,513266807,8da7ccee-3b8a-4cda-a74a-2f21fd5c12c3


¡Las filas 3309 y 3310 realmente están duplicadas! Elimino las filas duplicadas del dataset.

In [13]:
multistore_sales_df.drop_duplicates(inplace=True)
multistore_sales_df.shape

(5573721, 8)

### 2.2 Chequeo de valores faltantes

In [15]:
multistore_sales_df.isna().sum()

event_time       0
product_id       0
category_id      0
category_code    0
brand            0
price            0
user_id          0
user_session     3
dtype: int64

In [17]:
multistore_sales_df[multistore_sales_df.isna().any(axis=1)]

,event_time,product_id,category_id,category_code,brand,price,user_id,user_session
5294675,2020-04-16 15:39:34 UTC,5000313,2232732102950912587,appliances.sewing_machine,brother,175.81,563939005,NaN
5370972,2020-04-18 19:37:11 UTC,1005098,2232732093077520756,construction.tools.light,samsung,146.46,643633435,NaN
5581259,2020-04-27 07:18:28 UTC,2800477,2232732091307524418,appliances.kitchen.refrigerators,beko,422.12,514665829,NaN


Resulta que la columna *user_session* también tiene valores *NAs* y eliminamos las filas afectadas.

In [18]:
# Eliminamos estas filas con nulos
multistore_sales_df.dropna(inplace=True)

In [20]:
# Comprobación
multistore_sales_df[multistore_sales_df.isna().any(axis=1)]

,event_time,product_id,category_id,category_code,brand,price,user_id,user_session


## 3. Exportación de los datos a csv

In [22]:
multistore_sales_file_w = os.path.join(path + 'sales_multistore_10_2019_04_2020' + '.csv')
multistore_sales_df.to_csv(multistore_sales_file_w, index=False)